### Example execution of [FAIRsoft evaluation tool](https://openebench.bsc.es/observatory/Evaluation/) via [GitHub Metadata API](https://observatory.openebench.bsc.es/github-metadata-api/api-docs/) and [Observatory API](https://observatory.openebench.bsc.es/api/docs).

This notebook is intended to provide an example and validation for using the GitHub Metadata API and Observatory API to extract, score, evaluate and export research software metadata in a GitHub repository to improve research software FAIRness. There are other features of the FAIRsoft evaluation tool (e.g., metadata updating) that are not included in this notebook.

In [1]:
# import libaries to handle making API requests
import extractor_client
import metadata_client
import evaluation_client
import jsonld_client
import cff_client

The GitHub repository URL of the software is used as input to this workflow.  

In [2]:
github_url = 'https://github.com/abnerbog/hydroshare'

The GitHub repository URL can be input into the [/metadata-extractor-for-fairsoft/installation/id](https://observatory.openebench.bsc.es/github-metadata-api/api-docs/#/Metadata%20Extractor%20for%20FAIRsoft/get_metadata_extractor_for_fairsoft_installation_id) GitHub Metadata API endpoint which returns an installation ID for the [Metadata Extractor for FAIRsoft](https://github.com/settings/installations/54726444) GitHub App along with raw GitHub metadata (note the GitHub App must be installed in the user workspace with read access to the input repository):

In [3]:
install_id,owner,repo = extractor_client.get_installation_id(github_url)

Using this GitHub App installation ID, we can extract the repository metadata as JSON using the [/metadata-extractor-for-fairsoft/metadata](https://observatory.openebench.bsc.es/github-metadata-api/api-docs/#/Metadata%20Extractor%20for%20FAIRsoft/post_metadata) GitHub Metadata API endpoint:

In [4]:
metadata = metadata_client.get_repository_metadata(install_id,owner,repo)

In [5]:
print(metadata)

{'name': 'hydroshare', 'label': ['hydroshare'], 'description': [{'term': 'HydroShare is a collaborative website for better access to data and models in the hydrologic sciences. ', 'id': 0}], 'links': [], 'webpage': [{'term': 'https://www.hydroshare.org', 'id': 0}], 'isDisabled': False, 'isEmpty': False, 'isLocked': False, 'isPrivate': False, 'isTemplate': False, 'version': [], 'license': [{'term': {'name': 'BSD 3-Clause "New" or "Revised" License', 'url': 'http://choosealicense.com/licenses/bsd-3-clause/'}, 'id': 0}], 'repository': [{'term': 'https://github.com/abnerbog/hydroshare', 'id': 0}], 'topics': [], 'operations': [], 'authors': [{'term': {'name': 'Devin Cowan', 'type': 'person', 'email': 'devincowan@users.noreply.github.com', 'maintainer': False}, 'id': 0}, {'term': {'name': 'Devin Cowan', 'type': 'person', 'email': 'dcowan@cuahsi.org', 'maintainer': False}, 'id': 1}, {'term': {'name': 'Scott Black', 'type': 'person', 'email': 'sblack@cuahsi.org', 'maintainer': False}, 'id': 2}

The evaluation tool can be run by passing the metadata to the [/fair/evaluate](https://observatory.openebench.bsc.es/api/docs#/fair/evaluate_fair_evaluate_post) Observatory API endpoint to generate a JSON of FAIRsoft indicator scores and evaluation results (note [this documentation](https://inab.github.io/FAIRsoft_indicators/) contains detailed descriptions on each of the FAIRsoft indicators and scoring system):

In [6]:
fairsoft_scores,fairsoft_evaluation = evaluation_client.get_fairsoft_scores_and_evaluation(metadata)

In [7]:
print(fairsoft_scores)

{'name': 'hydroshare', 'type': '', 'version': None, 'F': 0.72, 'F1': 0.8, 'F1_1': True, 'F1_2': False, 'F2': 0.6, 'F2_1': True, 'F2_2': False, 'F3': 0.7, 'F3_1': False, 'F3_2': True, 'F3_3': False, 'A': 0.0, 'A1': 0.0, 'A1_1': False, 'A1_2': False, 'A1_3': False, 'A1_4': False, 'A1_5': False, 'A2': 0.0, 'A2_1': False, 'A2_2': False, 'A3': 0.0, 'A3_1': False, 'A3_2': False, 'A3_3': False, 'A3_4': False, 'A3_5': False, 'I': 0.0, 'I1': 0.0, 'I1_1': False, 'I1_2': False, 'I1_3': False, 'I1_4': False, 'I1_5': False, 'I2': 0.0, 'I2_1': False, 'I2_2': False, 'I3': 0.0, 'I3_1': False, 'I3_2': False, 'I3_3': False, 'R': 0.5, 'R1': 0.0, 'R1_1': False, 'R1_2': False, 'R2': 1.0, 'R2_1': True, 'R2_2': True, 'R3': 1.0, 'R3_1': False, 'R3_2': True, 'R4': 0.0, 'R4_1': False, 'R4_2': False, 'R4_3': False}


In [8]:
print(fairsoft_evaluation)

{'F1_1': ['The metadata is assigned a name to be identified.'], 'F1_2': ['❌ No version provided.', 'Result: FAILED'], 'F2_1': ['⚙️ Checking if any source provides structured metadata.', "🔍 Sources provided: ['github']", '⚙️ Comparing sources against: ', "✅ Source 'github' provides structured metadata.", 'Result: PASSED'], 'F2_2': ['⚙️ Verifying whether a topic or operation is part of a specific ontology or controlled vocabulary.', '🔍 No topics provided.', '🔍 No operations provided.', '❌ No topic or operation involves the use of ontologies or controlled vocabularies.', 'Result: FAILED'], 'F3_1': ['⚙️ Checking if any source matches software registries.', "🔍 Sources provided: ['github']", " ⚙️ Comparing sources against: ['biotools', 'bioconda', 'bioconductor', 'toolshed', 'sourceforge']", '❌ No source matches software registries. Checking registries ...', '⚙️ Checking if the specified software registry matches any of the known registries.', '🔍 No registries provided.', '⚙️ Comparing regis

We can also pass the metadata to the [/tools/jsonld](https://observatory.openebench.bsc.es/api/docs#/tools/tool_jsonld_tools_jsonld_post) Observatory API endpoint to generate a haromnized Schema.org metadata in JSON-LD format for adding to the GitHub repository.

In [9]:
jsonld_metadata = jsonld_client.get_jsonld_metadata(metadata)

In [10]:
print(jsonld_metadata)

{'@context': {'schema': 'https://schema.org/', 'bs': 'https://bioschemas.org/terms/', 'codemeta': 'https://w3id.org/codemeta/', 'rdf': 'https://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'https://www.w3.org/2000/01/rdf-schema#', 'maSMP': 'https://discovery.biothings.io/view/maSMP/'}, '@type': 'schema:SoftwareApplication', 'schema:name': 'hydroshare', '@id': 'https://openebench.bsc.es/bioschemas/tools/observatory:hydroshare:None/', 'schema:description': 'HydroShare is a collaborative website for better access to data and models in the hydrologic sciences. ', 'schema:license': [{'@type': 'schema:CreativeWork', 'schema:name': 'BSD 3-Clause "New" or "Revised" License', 'schema:url': 'http://choosealicense.com/licenses/bsd-3-clause/'}], 'schema:author': [{'@type': 'schema:Person', 'schema:name': 'Devin Cowan', 'schema:email': 'devincowan@users.noreply.github.com'}, {'@type': 'schema:Person', 'schema:name': 'Devin Cowan', 'schema:email': 'dcowan@cuahsi.org'}, {'@type': 'schema:Person', '

Lastly, the original JSON metadata can also be passed to the [/tools/cff](https://observatory.openebench.bsc.es/api/docs#/tools/tool_cff_tools_cff_post) Observatory API endpoint to generate a citation string in CFF format for adding to the GitHub repository.

In [11]:
cff_citation = cff_client.get_cff_citation(metadata)

In [12]:
print(cff_citation)

cff-version: 1.2.0
message: If you use this software, please cite it as below.
title: hydroshare
authors:
- given-names: Devin Cowan
  email: devincowan@users.noreply.github.com
- given-names: Devin Cowan
  email: dcowan@cuahsi.org
- given-names: Scott Black
  email: sblack@cuahsi.org
repository: https://github.com/abnerbog/hydroshare
url: https://www.hydroshare.org

